# StyleTTS2 Training Progress Monitor

This notebook helps you visualize and understand your StyleTTS2 training progress. You'll see loss curves, trend analysis, and guidance on what healthy training looks like.

## Key Questions This Answers:
- Is my model learning or is loss stuck?
- Am I overfitting (model memorizing instead of learning)?
- Are all loss components being trained?
- What does "good training" look like?

In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
from datetime import datetime

# Set style for better-looking plots
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

ModuleNotFoundError: No module named 'seaborn'

## Section 1: Load Training Data from Logs

We'll read your training log file and extract all loss metrics for analysis.

In [ ]:
# Configuration - Change these paths to match your setup
LOG_DIR = r"d:\Tesis\StyleTTS2\logs\angelina_es"
LOG_FILE = Path(LOG_DIR) / "train.log"

print(f"Loading training logs from: {LOG_FILE}")
print(f"Log file exists: {LOG_FILE.exists()}")

# Parse the training log
def parse_training_log(log_path):
    """
    Parse training log and extract metrics for each step.
    Returns a pandas DataFrame with all loss components.
    """
    data = {
        'epoch': [],
        'step': [],
        'total_steps': [],
        'mel_loss': [],
        'gen_loss': [],
        'disc_loss': [],
        'mono_loss': [],
        's2s_loss': [],
        'slm_loss': []
    }
    
    # Pattern: Epoch [1/200], Step [10/905], Mel Loss: 1.43579, Gen Loss: 0.00000, ...
    pattern = r'Epoch \[(\d+)/(\d+)\], Step \[(\d+)/(\d+)\], Mel Loss: ([\d.]+), Gen Loss: ([\d.]+), Disc Loss: ([\d.]+), Mono Loss: ([\d.]+), S2S Loss: ([\d.]+), SLM Loss: ([\d.]+)'
    
    with open(log_path, 'r') as f:
        for line in f:
            match = re.search(pattern, line)
            if match:
                epoch, total_epochs, step, total_steps, mel, gen, disc, mono, s2s, slm = match.groups()
                
                data['epoch'].append(int(epoch))
                data['step'].append(int(step))
                data['total_steps'].append(int(total_steps))
                data['mel_loss'].append(float(mel))
                data['gen_loss'].append(float(gen))
                data['disc_loss'].append(float(disc))
                data['mono_loss'].append(float(mono))
                data['s2s_loss'].append(float(s2s))
                data['slm_loss'].append(float(slm))
    
    df = pd.DataFrame(data)
    return df

# Load the data
df = parse_training_log(LOG_FILE)

print(f"\n✓ Successfully loaded {len(df)} training steps")
print(f"Epochs covered: {df['epoch'].min()} to {df['epoch'].max()}")
print(f"Latest epoch: {df['epoch'].iloc[-1]}")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# Calculate epoch-level statistics (average loss per epoch)
epoch_stats = df.groupby('epoch')[['mel_loss', 'gen_loss', 'disc_loss', 'mono_loss', 's2s_loss', 'slm_loss']].agg(['mean', 'min', 'max', 'std'])

print("Epoch-level statistics (first 10 epochs):")
print(epoch_stats.head(10))

## Section 2: Main Loss Trend (Per Epoch)

The most important metric: **Mel Loss**. This is your reconstruction loss - lower is better. In StyleTTS2:
- **Stage 1** (first ~30 epochs): Only Mel Loss is active. You should see it **decrease smoothly**.
- **Stage 2** (after ~30 epochs): Other losses activate (Gen Loss, Disc Loss, etc.). Mel Loss might increase slightly, which is normal.

Watch for:
- ✓ **Good**: Steady decrease in early training
- ✗ **Bad**: Loss stuck/flat for many epochs
- ✗ **Bad**: Sudden spikes (might indicate NaN or training issues)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Group by epoch and compute mean
epoch_mel = df.groupby('epoch')['mel_loss'].mean()

ax.plot(epoch_mel.index, epoch_mel.values, linewidth=2.5, label='Mel Loss', color='#1f77b4', marker='o', markersize=3)
ax.fill_between(epoch_mel.index, epoch_mel.values, alpha=0.2, color='#1f77b4')

ax.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax.set_ylabel('Mel Loss', fontsize=12, fontweight='bold')
ax.set_title('Training Progress: Mel Loss per Epoch\n(Main Reconstruction Loss)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

# Add annotations for first and last values
first_loss = epoch_mel.iloc[0]
last_loss = epoch_mel.iloc[-1]
improvement = (first_loss - last_loss) / first_loss * 100

ax.text(0.02, 0.98, f'First: {first_loss:.4f}\nLast: {last_loss:.4f}\nImprovement: {improvement:.1f}%', 
        transform=ax.transAxes, verticalalignment='top', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8), fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n📊 Mel Loss Summary:")
print(f"   Starting loss: {first_loss:.6f}")
print(f"   Current loss: {last_loss:.6f}")
print(f"   Improvement: {improvement:.2f}%")
print(f"   Trend: {'✓ IMPROVING' if improvement > 5 else '⚠ FLAT' if improvement < 1 else '✓ IMPROVING'}")

## Section 3: All Loss Components Across Training

StyleTTS2 uses multiple loss components. Let's see how each one changes:
- **Mel Loss**: Reconstruction (should decrease)
- **Gen Loss**: Generator adversarial loss
- **Disc Loss**: Discriminator loss
- **Mono Loss**: Monotonic alignment loss
- **S2S Loss**: Sequence-to-sequence loss
- **SLM Loss**: Speech Language Model loss

**Note**: Losses that are 0.0 mean they haven't been activated yet (depends on training stage).

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

loss_columns = ['mel_loss', 'gen_loss', 'disc_loss', 'mono_loss', 's2s_loss', 'slm_loss']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for idx, (col, color) in enumerate(zip(loss_columns, colors)):
    epoch_loss = df.groupby('epoch')[col].mean()
    
    axes[idx].plot(epoch_loss.index, epoch_loss.values, linewidth=2, color=color, marker='o', markersize=3)
    axes[idx].fill_between(epoch_loss.index, epoch_loss.values, alpha=0.2, color=color)
    axes[idx].set_title(col.replace('_', ' ').title(), fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Epoch', fontsize=10)
    axes[idx].set_ylabel('Loss', fontsize=10)
    axes[idx].grid(True, alpha=0.3)
    
    # Add text with current status
    has_data = (epoch_loss > 0).any()
    latest = epoch_loss.iloc[-1]
    status_text = f"Active\n(Latest: {latest:.6f})" if has_data and latest > 0 else "Not Active\n(0.0)"
    axes[idx].text(0.98, 0.97, status_text, transform=axes[idx].transAxes, 
                   verticalalignment='top', horizontalalignment='right',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7), fontsize=9)

plt.suptitle('All Loss Components During Training', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Summary table
print("\n📈 Loss Component Status:\n")
for col in loss_columns:
    epoch_loss = df.groupby('epoch')[col].mean()
    has_data = (epoch_loss > 0).any()
    if has_data:
        first_nonzero = epoch_loss[epoch_loss > 0].iloc[0] if (epoch_loss > 0).any() else np.nan
        first_epoch = epoch_loss[epoch_loss > 0].index[0] if (epoch_loss > 0).any() else 0
        current = epoch_loss.iloc[-1]
        print(f"  {col.upper():12} → Started at epoch {first_epoch:3d}, Current: {current:.6f}")
    else:
        print(f"  {col.upper():12} → Not yet activated (still 0.0)")

## Section 4: Training Stability & Variance

Stable training = loss stays relatively smooth with small changes between steps. Large spikes or noise indicate potential issues.

This chart shows **rolling average** (smooths out noise) and **variance** to see if training is stable.

In [ ]:
# Calculate rolling statistics for mel_loss (the main metric)
window_size = 50  # smooth over 50 steps

df['mel_loss_rolling_mean'] = df['mel_loss'].rolling(window=window_size, min_periods=1).mean()
df['mel_loss_rolling_std'] = df['mel_loss'].rolling(window=window_size, min_periods=1).std()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Loss with rolling average
ax1.plot(df.index, df['mel_loss'], alpha=0.3, label='Raw Mel Loss', color='lightblue', linewidth=0.8)
ax1.plot(df.index, df['mel_loss_rolling_mean'], label=f'Rolling Mean (window={window_size})', 
         color='darkblue', linewidth=2.5)
ax1.fill_between(df.index, 
                  df['mel_loss_rolling_mean'] - df['mel_loss_rolling_std'],
                  df['mel_loss_rolling_mean'] + df['mel_loss_rolling_std'],
                  alpha=0.2, color='darkblue', label='±1 Std Dev')
ax1.set_ylabel('Mel Loss', fontsize=11, fontweight='bold')
ax1.set_title('Training Loss: Raw vs Smoothed (Rolling Average)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Standard deviation (variance indicator)
ax2.plot(df.index, df['mel_loss_rolling_std'], color='#d62728', linewidth=2)
ax2.fill_between(df.index, df['mel_loss_rolling_std'], alpha=0.3, color='#d62728')
ax2.set_xlabel('Training Step', fontsize=11, fontweight='bold')
ax2.set_ylabel('Standard Deviation', fontsize=11, fontweight='bold')
ax2.set_title('Training Stability: Loss Variance (Lower = More Stable)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("\n🔍 Stability Analysis:\n")
avg_std = df['mel_loss_rolling_std'].mean()
recent_std = df['mel_loss_rolling_std'].iloc[-100:].mean() if len(df) > 100 else df['mel_loss_rolling_std'].mean()

print(f"  Average variance (all steps): {avg_std:.6f}")
print(f"  Recent variance (last 100 steps): {recent_std:.6f}")
print(f"  Stability trend: {'✓ STABLE' if recent_std < avg_std else '⚠ BECOMING LESS STABLE' if recent_std > avg_std * 1.5 else '✓ ACCEPTABLE'}")

## Section 5: Training Health Diagnosis

Here's a checklist to help you understand if training is going well:

In [ ]:
def diagnose_training(df, epoch_stats):
    """
    Analyze training and provide health diagnosis with actionable feedback.
    """
    print("=" * 70)
    print("🔬 TRAINING HEALTH DIAGNOSIS")
    print("=" * 70)
    
    # 1. Loss trend
    epoch_mel = df.groupby('epoch')['mel_loss'].mean()
    first_20 = epoch_mel.iloc[:min(20, len(epoch_mel))].mean() if len(epoch_mel) > 0 else 0
    last_20 = epoch_mel.iloc[max(0, len(epoch_mel)-20):].mean() if len(epoch_mel) > 0 else 0
    improvement = (first_20 - last_20) / first_20 * 100 if first_20 > 0 else 0
    
    print("\n✓ CHECK 1: Loss Decreasing?")
    if improvement > 5:
        print(f"  ✅ YES - Loss improved by {improvement:.1f}%")
        print(f"     First 20 epochs avg: {first_20:.6f}")
        print(f"     Last 20 epochs avg: {last_20:.6f}")
        print(f"     → GOOD: Model is learning!")
    elif improvement > 0:
        print(f"  ⚠️  SLIGHT - Loss improved by only {improvement:.1f}%")
        print(f"     → CAUTION: Very slow progress. Check learning rate.")
    else:
        print(f"  ❌ NO - Loss increased or flat by {-improvement:.1f}%")
        print(f"     → BAD: Model is not learning. Check:")
        print(f"        - Learning rate might be too high")
        print(f"        - Data quality/preprocessing")
        print(f"        - Model architecture")
    
    # 2. Stability check
    recent_loss = df['mel_loss'].iloc[-100:] if len(df) > 100 else df['mel_loss']
    loss_std = recent_loss.std()
    loss_mean = recent_loss.mean()
    noise_ratio = (loss_std / loss_mean) * 100 if loss_mean > 0 else 0
    
    print("\n✓ CHECK 2: Training Stable?")
    if noise_ratio < 10:
        print(f"  ✅ YES - Noise ratio: {noise_ratio:.1f}% (< 10%)")
        print(f"     → GOOD: Smooth, consistent training")
    elif noise_ratio < 20:
        print(f"  ⚠️  MODERATE - Noise ratio: {noise_ratio:.1f}% (10-20%)")
        print(f"     → ACCEPTABLE: Some fluctuation is normal")
    else:
        print(f"  ❌ HIGH VARIANCE - Noise ratio: {noise_ratio:.1f}% (> 20%)")
        print(f"     → CAUTION: Very noisy training. Consider:")
        print(f"        - Reducing batch size")
        print(f"        - Lowering learning rate")
        print(f"        - Checking for gradient clipping")
    
    # 3. Loss spike detection
    recent_50 = df['mel_loss'].iloc[-50:] if len(df) > 50 else df['mel_loss']
    max_recent = recent_50.max()
    min_recent = recent_50.min()
    spike_ratio = (max_recent - min_recent) / min_recent * 100 if min_recent > 0 else 0
    
    print("\n✓ CHECK 3: Sudden Spikes?")
    if spike_ratio < 30:
        print(f"  ✅ NO - Max spike in recent steps: {spike_ratio:.1f}%")
        print(f"     → GOOD: No NaN/Inf issues detected")
    elif spike_ratio < 100:
        print(f"  ⚠️  MINOR - Spike ratio: {spike_ratio:.1f}%")
        print(f"     → ACCEPTABLE: Small fluctuations are normal")
    else:
        print(f"  ❌ MAJOR SPIKES - Spike ratio: {spike_ratio:.1f}%")
        print(f"     → WARNING: Large sudden changes detected")
        print(f"     Max: {max_recent:.6f}, Min: {min_recent:.6f}")
    
    # 4. Training stage
    gen_loss = df.groupby('epoch')['gen_loss'].mean()
    gen_active = (gen_loss > 0).any()
    mel_epoch = epoch_mel.index.max()
    
    print("\n✓ CHECK 4: Training Stage?")
    if gen_active:
        gen_active_epoch = gen_loss[gen_loss > 0].index.min()
        print(f"  ℹ️  STAGE 2 ACTIVE")
        print(f"     Gen/Disc losses activated at epoch {gen_active_epoch}")
        print(f"     Current epoch: {mel_epoch}")
        print(f"     → Training adversarial objectives alongside reconstruction")
    else:
        print(f"  ℹ️  STAGE 1 ACTIVE")
        print(f"     Only Mel loss is active (no adversarial training yet)")
        print(f"     Current epoch: {mel_epoch}")
        print(f"     → Normal early-stage training")
    
    # 5. Overall assessment
    print("\n" + "=" * 70)
    print("📊 OVERALL ASSESSMENT")
    print("=" * 70)
    
    score = 0
    if improvement > 5: score += 1
    if noise_ratio < 20: score += 1
    if spike_ratio < 100: score += 1
    
    if score >= 2.5:
        print("✅ TRAINING IS HEALTHY - Keep going!")
        print("   Your model is learning well. Continue monitoring.")
    elif score >= 1.5:
        print("⚠️  TRAINING IS ACCEPTABLE - But watch for issues")
        print("   Training is progressing but has some concerns.")
    else:
        print("❌ TRAINING HAS ISSUES - Review settings")
        print("   Multiple problems detected. Consider changing hyperparameters.")
    
    print("\n" + "=" * 70)

# Run diagnosis
diagnose_training(df, epoch_stats)

## Section 6: Understanding Training Patterns

### What does "Good Training" look like?

**✅ Healthy Training Pattern:**
- **Mel Loss**: Decreases in early epochs (stage 1), then may increase slightly when other losses activate (stage 2) - this is expected
- **Smooth curve**: No sudden spikes or erratic behavior
- **Consistent improvement**: Week-to-week you see lower average losses
- **All losses active**: In stage 2, you should see Gen/Disc/Mono/S2S/SLM all > 0

**⚠️ Warning Signs:**
- **Flat loss**: No improvement for many epochs
- **Increasing loss**: Getting worse instead of better
- **Sudden spikes**: Loss suddenly jumps (could indicate numerical issues)
- **High variance**: Loss jumps around too much each step
- **Loss stays at 0**: Some losses not activating when they should be

### What's Normal?

1. **Stage 1 (first ~20-30 epochs)**: Only Mel Loss active - should see steady decrease
2. **Transition (epochs 30-40)**: Other losses activate. Mel Loss might increase slightly - THIS IS OK
3. **Stage 2 (epoch 40+)**: Multiple losses competing. Training becomes more complex but should stabilize

## Section 7: Detailed Epoch Comparison

Compare how losses change from one epoch to the next. This helps spot when things change (like when stage 2 starts).

In [ ]:
# Create detailed epoch summary table
epoch_summary = df.groupby('epoch').agg({
    'mel_loss': ['mean', 'min', 'max'],
    'gen_loss': 'mean',
    'disc_loss': 'mean',
    'mono_loss': 'mean',
    's2s_loss': 'mean',
    'slm_loss': 'mean'
}).round(6)

# Flatten column names
epoch_summary.columns = ['_'.join(col).strip() for col in epoch_summary.columns]
epoch_summary.columns = ['Mel_Avg', 'Mel_Min', 'Mel_Max', 'Gen', 'Disc', 'Mono', 'S2S', 'SLM']

# Show recent epochs
print("\n📋 Recent Epochs Summary (last 15 epochs):\n")
print(epoch_summary.tail(15).to_string())

# Identify when stage 2 starts (when non-mel losses activate)
stage2_epoch = None
for loss_col in ['gen_loss', 'disc_loss', 'mono_loss', 's2s_loss', 'slm_loss']:
    active_epochs = df[df[loss_col] > 0]['epoch'].unique()
    if len(active_epochs) > 0:
        first_active = active_epochs.min()
        if stage2_epoch is None or first_active < stage2_epoch:
            stage2_epoch = first_active

print(f"\n\n📌 Training Stages Detected:")
if stage2_epoch:
    print(f"  Stage 1 (Mel only): Epochs 1-{stage2_epoch-1}")
    print(f"  Stage 2 (Adversarial): Epochs {stage2_epoch}+")
else:
    print(f"  Only Stage 1 active (Mel loss only)")

## Section 8: Learning Rate & Convergence Analysis

Are you converging to a good solution? This shows if your model is finding better parameters each epoch.

In [ ]:
# Analyze convergence: how much does mel_loss change epoch to epoch?
epoch_mel = df.groupby('epoch')['mel_loss'].mean()
epoch_deltas = epoch_mel.diff().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Epoch-to-epoch changes (deltas)
colors_delta = ['green' if x < 0 else 'red' for x in epoch_deltas.values]
ax1.bar(epoch_deltas.index, epoch_deltas.values, color=colors_delta, alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax1.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax1.set_ylabel('Change in Mel Loss', fontsize=11, fontweight='bold')
ax1.set_title('Epoch-to-Epoch Loss Change\n(Negative = Improvement)', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Absolute change magnitude (convergence rate)
abs_deltas = epoch_deltas.abs()
ax2.plot(abs_deltas.index, abs_deltas.values, marker='o', color='#ff7f0e', linewidth=2, markersize=4)
ax2.fill_between(abs_deltas.index, abs_deltas.values, alpha=0.3, color='#ff7f0e')
ax2.set_xlabel('Epoch', fontsize=11, fontweight='bold')
ax2.set_ylabel('|Change|', fontsize=11, fontweight='bold')
ax2.set_title('Convergence Rate\n(Lower = More Stable/Converged)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analysis
print("\n📉 Convergence Analysis:\n")
early_changes = epoch_deltas.iloc[:min(20, len(epoch_deltas))].abs()
late_changes = epoch_deltas.iloc[max(0, len(epoch_deltas)-20):].abs()

print(f"  Early epochs (first 20) avg change: {early_changes.mean():.6f}")
print(f"  Recent epochs (last 20) avg change: {late_changes.mean():.6f}")

if late_changes.mean() < early_changes.mean() * 0.5:
    print(f"  ✅ CONVERGING - Changes getting smaller (good sign)")
elif late_changes.mean() < early_changes.mean():
    print(f"  ✓ SLOWLY CONVERGING - Some improvement")
else:
    print(f"  ⚠️  NOT CONVERGING - Changes not decreasing")
    print(f"     Your model might still be adjusting significantly each epoch")

# Improvement per epoch
total_improvement = (epoch_mel.iloc[0] - epoch_mel.iloc[-1]) / len(epoch_mel)
print(f"\n  Average improvement per epoch: {total_improvement:.6f}")
if total_improvement > 0.001:
    print(f"  ✓ GOOD - Meaningful progress each epoch")
elif total_improvement > 0:
    print(f"  ⚠️  SLOW - Very gradual progress")
else:
    print(f"  ✗ BAD - No net improvement")

## Section 9: Quick Reference Guide

### 🎯 What to Look For in Your Training

| Metric | Good ✅ | Bad ❌ |
|--------|--------|--------|
| **Mel Loss** | Decreases smoothly each epoch | Flat or increasing |
| **Loss Curve** | Smooth, consistent | Spiky, erratic jumps |
| **Variance** | Low (< 20% noise) | High (> 50% noise) |
| **Gen/Disc Loss** | Activates around epoch 20-40 | Stays 0.0 (not training) |
| **Stage Transition** | Mel increases slightly, then stabilizes | Mel crashes or diverges |
| **Recent Progress** | Still improving (last 20 epochs) | Plateaued for many epochs |

### ⚡ If Training is NOT Going Well...

1. **Loss not decreasing?**
   - Try reducing learning rate
   - Check data preprocessing
   - Verify batch size isn't too large

2. **Loss is spiky/unstable?**
   - Reduce learning rate
   - Increase batch size
   - Check for NaN in data

3. **Losses stuck at 0?**
   - Check config file (diff_epoch, joint_epoch parameters)
   - Verify training is in the right stage
   - Check if losses should be activated yet

4. **High variance early, stable later?**
   - This is normal! Early epochs are often noisier
   - As model converges, losses should smooth out

### 📊 Running This Monitor

- **After each training session**, run this notebook
- **Update the LOG_DIR** at the top if monitoring different model
- **Track weekly** to spot long-term trends
- **Watch for sudden changes** (might indicate config changes or code updates)